In [ ]:
pip install ragas datasets openai evaluate

In [ ]:
import os
import json
import numpy as np
from datasets import Dataset
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevance, context_precision, context_recall
# ragas.llms import RagasLLM 대신 문자열 이름으로 모델을 지정하면 Ragas가 자동으로 OpenAI 설정을 처리합니다.

# ----------------------------------------------------
# 0. 설정 및 환경 변수 확인
# ----------------------------------------------------
# OpenAI API 키가 환경 변수에 설정되어 있는지 확인
if "OPENAI_API_KEY" not in os.environ:
    # 예시를 위해 임시로 설정하거나 사용자에게 환경 변수 설정을 요구할 수 있습니다.
    # 실제 사용 시에는 터미널에서 export OPENAI_API_KEY='...' 와 같이 설정해야 합니다.
    print("WARNING: OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. 테스트를 위해 임시로 'dummy'를 사용하지만, 실제 API 호출은 실패합니다.")
    # raise ValueError("OPENAI_API_KEY 환경 변수를 설정해야 합니다.") # 실제 사용 시에는 이 라인을 활성화해야 합니다.

# GPT-4o-mini 모델 지정
GENERATOR_MODEL = "gpt-4o-mini"
CRITIC_MODEL = "gpt-4o-mini"
CONTEXT_FILE = "contexts.jsonl"
TEST_SIZE = 10  # 생성할 질문-답변 쌍의 개수 (테스트 목적)

# ----------------------------------------------------
# 1. JSONL 파일에서 Contexts 로드 및 Dataset으로 변환
# ----------------------------------------------------

def load_jsonl_to_list(file_path):
    """JSONL 파일을 읽어 딕셔너리 리스트로 반환"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def prepare_contexts_dataset():
    """Contexts.jsonl 파일을 로드하거나 생성하여 Ragas 입력 Dataset 형식으로 변환"""
    try:
        context_data_list = load_jsonl_to_list(CONTEXT_FILE)
        print(f"Loaded {len(context_data_list)} contexts from {CONTEXT_FILE}.")
    except FileNotFoundError:
        print(f"Error: {CONTEXT_FILE} 파일을 찾을 수 없습니다. 예시 파일을 생성합니다.")
        # 파일이 없을 경우 예시 데이터 생성 및 저장
        sample_data = [
            {"text": "지구는 태양계의 행성 중 하나이며, 생명체가 서식하는 유일한 곳이다.", "doc_id": "doc_1", "category": "space"},
            {"text": "물은 수소 원자 두 개와 산소 원자 하나로 구성되어 있으며, 100°C에서 끓는다.", "doc_2": "doc_2", "category": "chemistry"},
            {"text": "BERT는 구글이 개발한 언어 모델로, 양방향 학습을 통해 문맥 이해 능력을 향상시켰다.", "doc_3": "doc_3", "category": "ai"},
        ]
        with open(CONTEXT_FILE, 'w', encoding='utf-8') as f:
            for item in sample_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        context_data_list = sample_data
        print(f"Created sample {CONTEXT_FILE} with {len(context_data_list)} entries.")

    # Ragas 입력을 위한 Dataset 형식으로 변환 (각 텍스트 청크를 리스트로 감싸야 함)
    synthetic_contexts = Dataset.from_list([
        {'contexts': [item['text']],
         # 메타데이터를 활용하여 필터링 평가에 사용할 수 있도록 JSON 문자열로 저장
         'metadata': json.dumps({'doc_id': item.get('doc_id'), 'category': item.get('category')})}
        for item in context_data_list
    ])
    return synthetic_contexts

synthetic_contexts = prepare_contexts_dataset()

# ----------------------------------------------------
# 2. 합성 데이터셋 (테스트셋) 생성
# ----------------------------------------------------

print("\n--- Starting synthetic testset generation ---")

generator = TestsetGenerator.from_llm(
    generator_llm=GENERATOR_MODEL,
    critic_llm=CRITIC_MODEL,
    embeddings="openai" # 임베딩 모델도 OpenAI API를 사용
)

# 생성할 질문 유형 및 개수 설정
test_distributions = {
    simple: 0.5,           # 단순 질문
    reasoning: 0.3,        # 추론 질문
    multi_context: 0.2     # 다중 Context 질문
}

# 합성 데이터셋 생성
# 이 단계에서 LLM API 호출이 발생하며, 비용이 발생합니다.
test_set = generator.generate_with_text_chunks(
    synthetic_contexts,
    test_size=TEST_SIZE,
    distributions=test_distributions,
    raise_exceptions=False # 오류 발생 시 건너뛰기 설정
)

print(f"\n--- Synthetic Testset Generated ({len(test_set)} samples) ---")
print(test_set.to_pandas())


# ----------------------------------------------------
# 3. RAG 시스템으로 질문에 답변 (⭐ 사용자 정의 필수)
# ----------------------------------------------------
# 이 부분은 실제 RAG 시스템을 호출하여 답변을 얻는 과정입니다.
# 평가를 위해 RAG 시스템이 생성한 답변('answer')과 검색한 Contexts('contexts')가 필요합니다.

def generate_rag_answers(dataset):
    """
    (Placeholder) 실제 RAG 시스템을 호출하여 답변(answer)과 검색된 Contexts를 반환하는 함수
    """
    print("\n--- Simulating RAG Generation ---")

    answers = []
    retrieved_contexts = []

    for item in dataset:
        # 1. (실제 구현) item['question']을 RAG 시스템에 입력하고 답변을 얻습니다.
        # answer, retrieved_context = your_rag_system(item['question'])

        # 2. (예시) 정확도 100% 시뮬레이션을 위해, 정답(ground_truth)과 Contexts를 재사용합니다.
        answers.append(item['ground_truth'])
        retrieved_contexts.append(item['contexts'])

    return Dataset.from_dict({
        'question': dataset['question'],
        'answer': answers,                        # RAG 시스템이 생성한 답변
        'contexts': retrieved_contexts,           # RAG 시스템이 검색한 Contexts (리스트여야 함)
        'ground_truths': dataset['ground_truth']  # 합성 데이터셋의 정답
    })

# RAG 시스템의 답변 결과가 포함된 데이터셋
rag_evaluation_data = generate_rag_answers(test_set)
print("\n--- RAG Evaluation Data Prepared ---")


# ----------------------------------------------------
# 4. Ragas 지표를 이용한 평가 수행
# ----------------------------------------------------

print("\n--- Starting Ragas Evaluation ---")

# Ragas 평가 지표 정의
ragas_metrics = [
    faithfulness,        # 충실도: 답변이 검색된 Contexts에 기반하는 정도
    answer_relevance,    # 답변 관련성: 답변이 질문과 관련이 있는 정도
    context_precision,   # Context 정확도: 검색된 Contexts가 질문에 얼마나 정확한가
    context_recall       # Context 재현율: 검색된 Contexts가 정답 정보를 얼마나 잘 포함하는가
]

# 평가 수행 (LLM API 호출 발생 및 비용 발생)
results = evaluate(
    rag_evaluation_data,
    metrics=ragas_metrics,
    llm=GENERATOR_MODEL,
    embeddings="openai"
)

print("\n--- FINAL RAG EVALUATION RESULTS (Mean Scores) ---")
print(results)
# 결과 딕셔너리를 Pandas DataFrame으로 변환 후 평균 점수 출력
mean_scores = results.to_pandas().mean(numeric_only=True)
print(mean_scores.to_markdown())